In [ ]:
from google.colab import files
uploaded = files.upload()

Saving football_match.mp4 to football_match.mp4


In [ ]:
!pip install ultralytics supervision opencv-python-headless
#ultralytics -> to import model yolo. supervision -> to draw boundary easier. opencv - work without GPU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [ ]:
!pip install deep_sort_realtime  #deepsort -> to track the objects

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 83.2 MB/s eta 0:00:00


In [ ]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import supervision as sv

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
model = YOLO('yolov8s.pt')
video_path = "/content/football_match.mp4" #get a video and convert into frames
cap = cv2.VideoCapture(video_path)

tracker = DeepSort(max_age=30)

out = cv2.VideoWriter('tracked_output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (int(cap.get(3)), int(cap.get(4)))) #30 - frames , height, width

while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
      break

    results = model(frame)[0]

    detections = []
    for r in results.boxes:
      #to draw an bounding box
      x1, y1, x2, y2 = map(int, r.xyxy[0])  #draw doundaries
      conf = float(r.conf[0])  #calculte confidence score
      cls = int(r.cls[0])
      if cls == 0:  #class 0 = 'person' in coco
        detections.append(([x1,y1,x2 - x1, y2-y1], conf, 'player'))

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
      if not track.is_confirmed():
        continue
      track_id = track.track_id
      l, t, w, h = track.to_ltrb()
      cv2.rectangle(frame, (int(l), int(t)), (int(w), int(h)), (0, 255, 0), 2) #color , thickness
      cv2.putText(frame, f"ID: {track_id}", (int(l), int(t) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0),2) #font-scale(size of text), color, thickness
    out.write(frame)

cap.release()
out.release()


0: 384x640 11 persons, 18.4ms
Speed: 1.9ms preprocess, 18.4ms inference, 7.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 10.8ms
Speed: 2.3ms preprocess, 10.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 11.5ms
Speed: 1.6ms preprocess, 11.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 1 sports ball, 10.8ms
Speed: 1.4ms preprocess, 10.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 1 sports ball, 11.1ms
Speed: 1.8ms preprocess, 11.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 10.9ms
Speed: 1.7ms preprocess, 10.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 11.2ms
Speed: 1.7ms preprocess, 11.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 10.9ms
Speed: 1.9ms preprocess, 10.9ms inference, 1.

In [ ]:
from google.colab import files
files.download("tracked_output.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>